In [5]:
import os
import glob

In [1]:
import pandas as pd

In [2]:
import config

In [3]:
config.fp_ds_answers

'/home/colin/code/bq_snowflake_benchmark/ds/v2.11.0rc2/answer_sets'

In [9]:
files = glob.glob(config.fp_ds_answers + config.sep + "*.ans")
data = []
for fp in files:
    file_name = os.path.basename(fp)
    data.append([file_name, fp])
dffp = pd.DataFrame(data, columns=["fp", "ffp"])

In [10]:
dffp

,fp,ffp
0,62_NULLS_FIRST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
1,62_NULLS_LAST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
2,29.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
3,22_NULLS_LAST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
4,83.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
...,...,...
124,95.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
125,79_NULLS_FIRST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
126,75.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....
127,23_NULLS_FIRST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....


In [20]:
x = dffp.loc[0, "fp"]
x

'62_NULLS_FIRST.ans'

In [19]:
y = dffp.loc[2, "fp"]
y

'29.ans'

In [21]:
x.split(".")[0]

'62_NULLS_FIRST'

In [50]:
def f(fname):
    x = fname.split(".")
    y = x[0]
    z = y.split("_")
    #print(x, y, z)
    a = len(z)
    #print(a)
    n = z[0]
    #print(n)
    locate = "NA"
    if a == 3:
        locate = z[2]
    return n, locate

In [48]:
f(y)

NA


('29', 'NA')

In [49]:
f(x)

FIRST


('62', 'FIRST')

In [51]:
dffp.fp.apply(f)

0      (62, FIRST)
1       (62, LAST)
2         (29, NA)
3       (22, LAST)
4         (83, NA)
          ...     
124       (95, NA)
125    (79, FIRST)
126       (75, NA)
127    (23, FIRST)
128       (50, NA)
Name: fp, Length: 129, dtype: object

In [59]:
def f(fname):
    y = fname.split(".")[0]
    z = y.split("_")
    return y, z

In [60]:
def fn(fname):
    _, z = f(fname)
    n = z[0]
    return n

In [61]:
def fnull(fname):
    _, z = f(fname)
    locate = "NA"
    if len(z) == 3:
        locate = z[2]
    return locate

In [62]:
dffp["fn"] = dffp.fp.apply(fn)

In [64]:
dffp["fnull"] = dffp.fp.apply(fnull)

In [65]:
dffp.head()

,fp,ffp,fn,fnull
0,62_NULLS_FIRST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,62,FIRST
1,62_NULLS_LAST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,62,LAST
2,29.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,29,NA
3,22_NULLS_LAST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,22,LAST
4,83.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,83,NA


In [73]:
dffp.fn = dffp.fn.astype(int)

In [79]:
dffp = dffp.sort_values(by=["fn"], axis=0)

In [81]:
dffp.reset_index(inplace=True, drop=True)

In [82]:
dffp.head()

,fp,ffp,fn,fnull
0,1.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,1,NA
1,2.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,2,NA
2,3.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,3,NA
3,4.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,4,NA
4,5_NULLS_FIRST.ans,/home/colin/code/bq_snowflake_benchmark/ds/v2....,5,FIRST


In [94]:
dffp.loc[1, "ffp"]

'/home/colin/code/bq_snowflake_benchmark/ds/v2.11.0rc2/answer_sets/2.ans'

In [130]:
header = ""
data = []
with open(dffp.loc[0, "ffp"], "r") as f:
    c = 0
    for line in f:
        if c == 0:
            header = line
            c += 1
        elif c == 1:
            c += 1
        elif line == "\n":
            break
            print("HERE >> ", c)
        else:
            line_data = line.strip().split("\t")
            if len(line_data) == 1:
                line_data = line_data[0]
            data.append(line_data)
            c += 1
header = header.strip().split("\t")
print(header)

['C_CUSTOMER_ID']


In [142]:
def parse_ans(fp):
    """Parse the answer files to the TPC-DS validiation/qualification
    1GB dataset and queries
    
    Parameters
    ----------
    fp : str, absolute path to answer file, with .ans file extension
    
    Returns
    -------
    header : list of str, column names
    data : list of list, data from tab delimited data
    """
    header = ""
    data = []
    with open(fp, "r") as f:
        c = 0
        for line in f:
            if c == 0:
                header = line
                c += 1
            elif c == 1:
                c += 1
            elif line == "\n":
                break
                print("HERE >> ", c)
            else:
                line_data = line.strip().split(" ")
                if len(line_data) == 1:
                    line_data = line_data[0]
                data.append(line_data)
                c += 1
    header = header.strip().split("\t")
    return header, data

In [143]:
h, d = parse_ans(dffp.loc[2, "ffp"])

In [144]:
d

[['1998',
  '',
  '',
  '',
  '2001001',
  'amalgimporto',
  '#1\t\t\t\t\t',
  '',
  '',
  '45162.45'],
 ['1998',
  '',
  '',
  '',
  '5003001',
  'exportischolar',
  '#1',
  '\t\t\t\t',
  '',
  '',
  '40600.56'],
 ['1998',
  '',
  '',
  '',
  '5001001',
  'amalgscholar',
  '#1\t\t\t\t\t',
  '',
  '',
  '32515.11'],
 ['1998',
  '',
  '',
  '',
  '3004001',
  'edu',
  'packexporti',
  '#1\t\t\t\t',
  '',
  '',
  '32443.16'],
 ['1998',
  '',
  '',
  '',
  '3001001',
  'amalgexporti',
  '#1\t\t\t\t\t',
  '',
  '',
  '30280.72'],
 ['1998',
  '',
  '',
  '',
  '3001002',
  'amalgexporti',
  '#2\t\t\t\t\t',
  '',
  '',
  '27644.68'],
 ['1998',
  '',
  '',
  '',
  '1003001',
  'exportiamalg',
  '#1\t\t\t\t\t',
  '',
  '',
  '',
  '25408.1'],
 ['1998',
  '',
  '',
  '',
  '7007010',
  'brandbrand',
  '#10\t\t\t\t\t',
  '',
  '',
  '23346.45'],
 ['1998',
  '',
  '',
  '',
  '8001009',
  'amalgnameless',
  '#9\t\t\t\t\t',
  '',
  '',
  '20792.57'],
 ['1998',
  '',
  '',
  '10010002',
  'univamal

In [123]:
data[0:5]

['AAAAAAAAAAABBAAA',
 'AAAAAAAAAAADBAAA',
 'AAAAAAAAAAADBAAA',
 'AAAAAAAAAAAKAAAA',
 'AAAAAAAAAABDAAAA']

In [131]:
header

['C_CUSTOMER_ID']

In [132]:
pd.DataFrame(data, columns=header)

,C_CUSTOMER_ID
0,AAAAAAAAAAABBAAA
1,AAAAAAAAAAADBAAA
2,AAAAAAAAAAADBAAA
3,AAAAAAAAAAAKAAAA
4,AAAAAAAAAABDAAAA
...,...
95,AAAAAAAAACBDAAAA
96,AAAAAAAAACBEBAAA
97,AAAAAAAAACBNAAAA
98,AAAAAAAAACBPAAAA


In [90]:
x = "----------------"



In [91]:
set(x)

{'-'}

In [92]:
set(" ")

{' '}